In [1]:
%cd ~/repo/protein-transfer

/home/t-fli/repo/protein-transfer


In [2]:
%load_ext blackcellmagic

In [5]:
import os
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv("data/structure/secondary_structure/tape_ss3.csv")
df["ss3"] = df["ss3"].apply(lambda x: np.array(x[1:-1].split(", ")))
df.head()

,sequence,ss3,split
0,AETVESCLAKSHTENSFTNVXKDDKTLDRYANYEGCLWNATGVVVC...,"[2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1, 1, 1, ...",train
1,ASQEISKSIYTCNDNQVXEVIYVNTEAGNAYAIISQVNEXIPXRLX...,"[2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, ...",train
2,XGSSHHHHHHSSGRENLYFQGXNISEINGFEVTGFVVRTTNADEXN...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",train
3,SNADYNRLSVPGNVIGKGGNAVVYEDAEDATKVLKMFTTSQSNEEV...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, ...",train
4,DTVGRPLPHLAAAMQASGEAVYCDDIPRYENELFLRLVTSTRAHAK...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, ...",train


In [7]:
df.sequence[0], df.ss3[0], 

('AETVESCLAKSHTENSFTNVXKDDKTLDRYANYEGCLWNATGVVVCTGDETQCYGTWVPIGLAIPENEGGGSEGGGSEGGGSEGGGTKPPEYGDTPIPGYTYINPLDGTYPPGTEQNPANPNPSLEESQPLNTFMFQNNRFRNRQGALTVYTGTVTQGTDPVKTYYQYTPVSSKAMYDAYWNGKFRDCAFHSGFNEDIFVCEYQGQSSDLPQPPVNA',
 array(['2', '2', '2', '0', '0', '0', '0', '0', '2', '2', '2', '2', '1',
        '1', '1', '1', '1', '1', '2', '2', '1', '1', '2', '2', '2', '2',
        '2', '2', '1', '1', '1', '1', '1', '2', '2', '1', '1', '1', '1',
        '1', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2', '2', '1',
        '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2',
        '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2',
        '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2',
        '2', '2', '2', '2', '2', '1', '1', '1', '1', '1', '2', '2', '2',
        '2', '2', '2', '2', '2', '2', '2', '1', '2', '2', '2', '2', '1',
        '2', '2', '2', '2', '2', '2', '1', '1', '1', '2', '2', '2', '2',
        '2', '2', '2', '1', '1', '1', '2', '2', 

In [19]:
df.sequence[:4].shape, df.ss3[:4].shape, len(df.sequence[1]), len(df.ss3[1])

((4,), (4,), 98, 98)

In [8]:
df.split.unique()

array(['train', 'valid', 'cb513', 'ts115', 'casp12'], dtype=object)

In [3]:
from scr.preprocess.data_process import std_ssdf

In [4]:
import os
import numpy as np
import pandas as pd

def std_ssdf(
    ssdf_path: str = "data/structure/secondary_structure/tape_ss3.csv",
) -> None:
    """
    A function that standardize secondary structure dataset
    to set up as columns as sequence, target, set, validation
    where set is train or test and add validation as true
    """
    folder_path = os.path.dirname(ssdf_path)

    df = pd.read_csv(ssdf_path)
    
    # add validation column
    df["validation"] = df["split"].apply(lambda x: True if x == "valid" else np.nan)
    # now replace valid to train
    df["split"] = df["split"].replace("valid", "train")
    # rename all columns
    df.columns = ["sequence", "target", "set", "validation"]

    # get all kinds of test sets
    ss_tests = set(df["set"].unique()) - set(["train"])

    for ss_test in ss_tests:
        ss_df = df.loc[~df["set"].isin(set(ss_tests) - set([ss_test]))]
        ss_df["set"] = ss_df["set"].replace(ss_test, "test")
        display(ss_df.head())
        display(ss_df.target[0])
        ss_df.to_csv(os.path.join(folder_path, ss_test + ".csv"), index=False)


<function scr.preprocess.data_process.std_ssdf(ssdf_path: 'str' = 'data/structure/secondary_structure/tape_ss3.csv') -> 'None'>

In [5]:
std_ssdf()

In [41]:
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
import numpy as np

X, y1 = make_classification(n_samples=10, n_features=100,
                            n_informative=30, n_classes=3,
                            random_state=1)
y2 = shuffle(y1, random_state=1)
y3 = shuffle(y1, random_state=2)
Y = np.vstack((y1, y2, y3)).T
n_samples, n_features = X.shape # 10,100
n_outputs = Y.shape[1] # 3
n_classes = 3
forest = RandomForestClassifier(random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=2)
multi_target_forest.fit(X, Y).predict(X)

array([[2, 2, 0],
       [1, 2, 1],
       [2, 1, 0],
       [0, 0, 2],
       [0, 2, 1],
       [0, 0, 2],
       [1, 1, 0],
       [1, 1, 1],
       [0, 0, 2],
       [2, 0, 0]])

In [9]:
from scr.encoding.encoding_classes import ESMEncoder

In [10]:
mean_batch_encoder = ESMEncoder(
    encoder_name="esm1_t6_43M_UR50S",
).encode(mut_seqs=list(df.sequence), batch_size=4, flatten_emb=False)
one_mean_emb = next(mean_batch_encoder)
one_mean_emb[0].shape

Generating esm1_t6_43M_UR50S upto 6 layer embedding ...


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main
  0%|          | 0/2875 [00:00<?, ?it/s]

No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening
No embedding flattening


(4, 217, 768)

In [11]:
X = one_mean_emb[0]
X.shape

(4, 217, 768)

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [13]:
df.ss3[:4]

0    [2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1, 1, 1, ...
1    [2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, ...
2    [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...
3    [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, ...
Name: ss3, dtype: object

In [14]:
Y = df.ss3[:4].to_numpy()
Y.shape

(4,)

In [15]:
Y

array([array(['2', '2', '2', '0', '0', '0', '0', '0', '2', '2', '2', '2', '1',
              '1', '1', '1', '1', '1', '2', '2', '1', '1', '2', '2', '2', '2',
              '2', '2', '1', '1', '1', '1', '1', '2', '2', '1', '1', '1', '1',
              '1', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2', '2', '1',
              '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2',
              '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2',
              '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2',
              '2', '2', '2', '2', '2', '1', '1', '1', '1', '1', '2', '2', '2',
              '2', '2', '2', '2', '2', '2', '2', '1', '2', '2', '2', '2', '1',
              '2', '2', '2', '2', '2', '2', '1', '1', '1', '2', '2', '2', '2',
              '2', '2', '2', '1', '1', '1', '2', '2', '1', '1', '1', '1', '1',
              '1', '2', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
              '1', '2', '2', '2', '2', '2', '1', '1'

In [21]:
X.shape[1]

217

In [22]:
np.stack([np.pad(y, pad_width=(0, X.shape[1] - len(y)), constant_values=-1) for y in Y])

array([['2', '2', '2', '0', '0', '0', '0', '0', '2', '2', '2', '2', '1',
        '1', '1', '1', '1', '1', '2', '2', '1', '1', '2', '2', '2', '2',
        '2', '2', '1', '1', '1', '1', '1', '2', '2', '1', '1', '1', '1',
        '1', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2', '2', '1',
        '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2',
        '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2',
        '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2',
        '2', '2', '2', '2', '2', '1', '1', '1', '1', '1', '2', '2', '2',
        '2', '2', '2', '2', '2', '2', '2', '1', '2', '2', '2', '2', '1',
        '2', '2', '2', '2', '2', '2', '1', '1', '1', '2', '2', '2', '2',
        '2', '2', '2', '1', '1', '1', '2', '2', '1', '1', '1', '1', '1',
        '1', '2', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
        '1', '2', '2', '2', '2', '2', '1', '1', '1', '1', '1', '1', '1',
        '1', '2', '2', '2', '0', '0', '0', '0', '0'

In [ ]:
# pad wiht -1
np.pad(y, pad_width=((0, X.shape[1] - len(y)), (-1 , -1)))